In [1]:
import tensorflow as tf
tf.__version__

'2.4.0'

# Load the training data

In [6]:
# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset

train_dataset = get_dataset(
    #"H:/MachineLearningPractice/SparrowRecSys/NewCode/JavaPart/src/main/resources/webroot/sampledata/trainingSamples.csv"
    r"D:\MachineLearningPractice\SparrowRecSys\NewCode\JavaPart\src\main\resources\webroot\sampledata\trainingSamples.csv"
)
test_dataset = get_dataset(
#     "H:/MachineLearningPractice/SparrowRecSys/NewCode/JavaPart/src/main/resources/webroot/sampledata/testSamples.csv"
    r"D:\MachineLearningPractice\SparrowRecSys\NewCode\JavaPart\src\main\resources\webroot\sampledata\testSamples.csv"
)

# Config

In [7]:
# Config
RECENT_MOVIES = 5  # userRatedMovie{1-5}
EMBEDDING_SIZE = 10

# define input for keras model

In [8]:
inputs = {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),
    'userRatedMovie2': tf.keras.layers.Input(name='userRatedMovie2', shape=(), dtype='int32'),
    'userRatedMovie3': tf.keras.layers.Input(name='userRatedMovie3', shape=(), dtype='int32'),
    'userRatedMovie4': tf.keras.layers.Input(name='userRatedMovie4', shape=(), dtype='int32'),
    'userRatedMovie5': tf.keras.layers.Input(name='userRatedMovie5', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
}

# movie id embedding feature

In [9]:
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, EMBEDDING_SIZE)

# user id embedding feature

In [10]:
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, EMBEDDING_SIZE)

# embed the genre

In [11]:
# genre features vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']
# user genre embedding feature
user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="userGenre1",
                                                                           vocabulary_list=genre_vocab)
user_genre_emb_col = tf.feature_column.embedding_column(user_genre_col, EMBEDDING_SIZE)
# item genre embedding feature
item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="movieGenre1",
                                                                           vocabulary_list=genre_vocab)
item_genre_emb_col = tf.feature_column.embedding_column(item_genre_col, EMBEDDING_SIZE)

# user behaviors

In [12]:
recent_rate_col = [
    tf.feature_column.numeric_column(key='userRatedMovie1', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie2', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie3', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie4', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie5', default_value=0),
]

# user profile

In [13]:
user_profile = [
    user_emb_col,
    user_genre_emb_col,
    tf.feature_column.numeric_column('userRatingCount'),
    tf.feature_column.numeric_column('userAvgRating'),
    tf.feature_column.numeric_column('userRatingStddev'),
]

# context features

In [14]:
context_features = [
    item_genre_emb_col,
    tf.feature_column.numeric_column('releaseYear'),
    tf.feature_column.numeric_column('movieRatingCount'),
    tf.feature_column.numeric_column('movieAvgRating'),
    tf.feature_column.numeric_column('movieRatingStddev'),
]

# Build the network structure

In [16]:
user_profile_layer = tf.keras.layers.DenseFeatures(user_profile)(inputs)
user_behaviors_layer = tf.keras.layers.DenseFeatures(recent_rate_col)(inputs)
candidate_emb_layer = tf.keras.layers.DenseFeatures([movie_emb_col])(inputs)
context_features_layer = tf.keras.layers.DenseFeatures(context_features)(inputs)

In [17]:
# Activation Unit
user_behaviors_emb_layer = tf.keras.layers.Embedding(input_dim=1001,
                                                     output_dim=EMBEDDING_SIZE,
                                                     mask_zero=True)(user_behaviors_layer)  # mask zero
repeated_candidate_emb_layer = tf.keras.layers.RepeatVector(RECENT_MOVIES)(candidate_emb_layer)

In [18]:
activation_sub_layer = tf.keras.layers.Subtract()([user_behaviors_emb_layer,
                                                   repeated_candidate_emb_layer])  # element-wise sub
activation_product_layer = tf.keras.layers.Multiply()([user_behaviors_emb_layer,
                                                       repeated_candidate_emb_layer])  # element-wise product

In [19]:
activation_all = tf.keras.layers.concatenate([activation_sub_layer, user_behaviors_emb_layer,
                                              repeated_candidate_emb_layer, activation_product_layer], axis=-1)

In [20]:
activation_unit = tf.keras.layers.Dense(32)(activation_all)
activation_unit = tf.keras.layers.PReLU()(activation_unit)
activation_unit = tf.keras.layers.Dense(1, activation='sigmoid')(activation_unit)
activation_unit = tf.keras.layers.Flatten()(activation_unit)
activation_unit = tf.keras.layers.RepeatVector(EMBEDDING_SIZE)(activation_unit)
activation_unit = tf.keras.layers.Permute((2, 1))(activation_unit)
activation_unit = tf.keras.layers.Multiply()([user_behaviors_emb_layer, activation_unit])

In [22]:
# sum pooling
user_behaviors_pooled_layers = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(activation_unit)

# fc layer
concat_layer = tf.keras.layers.concatenate([user_profile_layer, user_behaviors_pooled_layers,
                                            candidate_emb_layer, context_features_layer])
output_layer = tf.keras.layers.Dense(128)(concat_layer)
output_layer = tf.keras.layers.PReLU()(output_layer)
output_layer = tf.keras.layers.Dense(64)(output_layer)
output_layer = tf.keras.layers.PReLU()(output_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(output_layer)

model = tf.keras.Model(inputs, output_layer)

# Train the model

In [23]:
# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

In [24]:
# train the model
model.fit(train_dataset, epochs=5)

Epoch 1/5


c:\users\xmk23\appdata\local\continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\functional.py:595: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


7403/7403 [==============================] - 81s 10ms/step - loss: 7.6831 - accuracy: 0.5641 - auc: 0.5698 - auc_1: 0.6136887 - accuracy: 0.5630 - auc:
Epoch 2/5
7403/7403 [==============================] - 73s 10ms/step - loss: 1.2894 - accuracy: 0.6219 - auc: 0.6486 - auc_1: 0.6856
Epoch 3/5
7403/7403 [==============================] - 67s 9ms/step - loss: 0.6268 - accuracy: 0.6905 - auc: 0.7400 - auc_1: 0.7632
Epoch 4/5
7403/7403 [==============================] - 67s 9ms/step - loss: 0.5292 - accuracy: 0.7389 - auc: 0.8070 - auc_1: 0.8273
Epoch 5/5
7403/7403 [==============================] - 65s 9ms/step - loss: 0.4996 - accuracy: 0.7574 - auc: 0.8317 - auc_1: 0.8535


In [25]:
# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))


1870/1870 [==============================] - 12s 4ms/step - loss: 0.6092 - accuracy: 0.6907 - auc: 0.7453 - auc_1: 0.7683


Test Loss 0.6091870665550232, Test Accuracy 0.6906862854957581, Test ROC AUC 0.745329737663269, Test PR AUC 0.7682846784591675


In [26]:
# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))

Predicted good rating: 61.91%  | Actual rating label:  Good Rating
Predicted good rating: 43.49%  | Actual rating label:  Bad Rating
Predicted good rating: 71.38%  | Actual rating label:  Good Rating
Predicted good rating: 19.54%  | Actual rating label:  Bad Rating
Predicted good rating: 55.49%  | Actual rating label:  Good Rating
Predicted good rating: 51.82%  | Actual rating label:  Good Rating
Predicted good rating: 52.17%  | Actual rating label:  Good Rating
Predicted good rating: 76.92%  | Actual rating label:  Good Rating
Predicted good rating: 23.35%  | Actual rating label:  Bad Rating
Predicted good rating: 64.95%  | Actual rating label:  Good Rating
Predicted good rating: 0.97%  | Actual rating label:  Bad Rating
Predicted good rating: 55.35%  | Actual rating label:  Good Rating
